In [1]:
from __future__ import print_function
from fenics import *
from mshr import *
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import vtk

In [3]:
import sympy as sym

so we have family of functions $q_N(x,y) =1+x^N+y^N$ and the function $u = sin(\pi y) cos(\pi x)$, mesh of size $meshN \times meshN$

In [4]:
def q(x, y, degree=1):
    return 1+ x**degree +y**degree
def boundary(x, on_boundary):
    return on_boundary

def pair_generator(N, meshN = 10):
    #this is for 
    x, y = sym.symbols('x[0], x[1]')
    u = sym.cos(sym.pi * x) *   sym.sin(sym.pi * y)
    f = - sym.diff(q(x,y, 2)*sym.diff(u, x), x) - sym.diff(q(x,y, 2)*sym.diff(u, y), y)
    f = sym.simplify(f)
    u_code = sym.printing.ccode(u)
    f_code = sym.printing.ccode(f)
    
    
    a = 'pow(x[0],{})+1+pow(x[1],{})'.format(N,N)
    kappa = Expression(a, degree=1)
    
    
    mesh = UnitSquareMesh(10, 10)
    V = FunctionSpace(mesh, 'P', 1)

# HAVE NO CLUE WHY WE ARE DOING  THAT
    u_D = Expression(u_code, degree=1)
    f = Expression(f_code, degree=2)

    
#we have direchlet conditions in BERG NYSTROM paper
    bc = DirichletBC(V, u_D, boundary)
    u = Function(V)  # Note: not TrialFunction!
    v = TestFunction(V)

    F = kappa*dot(grad(u), grad(v))*dx - f*v*dx

# Compute solution
    solve(F == 0, u, bc)
    arr_u = u.compute_vertex_values(mesh)
    arr_kappa = kappa.compute_vertex_values(mesh)
    return arr_u, arr_kappa
    

In [5]:
data_u = {}
data_kappa = {}
N = [i for i in range(5)]
for i in N:
    u, kappa = pair_generator(i)
    data_u[i] = u
    data_kappa[i] = kappa
    